In [1]:
using Iterators
using DataStructures
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util"                                                        

In [2]:
using Packing

In [3]:
using DataStructures

function Base.sum(acc::Accumulator)
    sum(values(acc.map))
end

function Base.sum(acc::Dict)
    sum(values(acc))
end

function freq2prob{T,V<:Number}(acc::Union{Accumulator{T,V},Dict{T,V}})
    
    ret=Dict{T,Float64}()
    total = sum(acc)
    for (k,v) in acc
        ret[k]=v/total
    end
    ret
end

freq2prob (generic function with 1 method)

In [4]:
function clean(ss)
    @pipe (ss 
    |> replace(_, r"\[.*?\] ?","")  #Remove Nonword sounds
    |> replace(_, r"\<.*?\> ","")  #Remove Verbal Deltions
    |> replace(_, r"\*(.*?)\*",s"\1") #Remove mispronounciation marks
    |> replace(_,r"\:|\-\s\.\s\-", "") #remove intraword pauses
    |> replace(_,r"\w+\- ","") #remove stuttered words
    |> replace(_, r"[!\.,\?]","")        #Remove punctation as it is not used traditionally (see sro spec)
    
    |> replace(_, r"\s+",' ') #Remove repeated spaces
    #|> replace(_, r"([A-Z])\s([A-Z])", s"\1\2") #Merge len(2) abbrev
    |> replace(_, r".*[~\(\)\-\<\>#'].*","")#Remove everything if anything unfixable found
    |> lowercase
    |> strip 
    )
end


clean (generic function with 1 method)

In [5]:
const START_MARKER = "**START**"
const END_MARKER = "**END**"

"**END**"

In [6]:
path="../../Resources/corpora/atis2_text/"
function valid(ss)
    typeof(ss) <: ASCIIString  && length(ss)>0
end
    
corpus = @pipe readdir(path) |> filter!(fn -> splitext(fn)[2]==".sro", _) |> map(_) do fn
    try open(readall, path*fn) end
    end |> filter!(valid,_) |> map(clean,_) |> filter!(valid,_) |> map(s->split(s),_);
corpus_vocab = @pipe corpus |> map(Set,_) |> reduce(union,_)
length(corpus_vocab)

1131

In [7]:
using WordEmbeddings
LL, word_indexes, indexed_words = load_word2vec_embeddings("word_emb_data/GoogleNews-vectors-negative300.bin", length(corpus_vocab), corpus_vocab);

In [8]:
setdiff(corpus_vocab,indexed_words)


16-element Array{UTF8String,1}:
 "and"      
 "a"        
 "respeak"  
 "nondirect"
 "maluso"   
 "of"       
 "panam"    
 "stapleton"
 "fokker"   
 "lufthansa"
 "to"       
 "laguardia"
 "nonjets"  
 "hartfield"
 "mcdonnell"
 "dulles"   

In [9]:
#Kind of the opposite of a stop word. This word has little meaning (So zero value), but much structural importance
forcewords = ["and", "a", "of", "to"]
for word in forcewords
    @assert(!(word in indexed_words))
    push!(indexed_words, word)
    word_indexes[word] = length(indexed_words)
end
LL = [LL zeros(size(LL,1),length(forcewords))]

300x1119 Array{Float64,2}:
  0.0529562  -0.00851202  -0.0123606    …  -0.0548293   0.0  0.0  0.0  0.0
  0.0654598  -0.0342245   -0.0222299        0.0239878   0.0  0.0  0.0  0.0
  0.0661953   0.0322839    0.0655398       -0.0190492   0.0  0.0  0.0  0.0
  0.0470722   0.0458679    0.0394772        0.0826471   0.0  0.0  0.0  0.0
  0.0522207  -0.0131429   -0.0866199        0.0620861   0.0  0.0  0.0  0.0
 -0.0820086  -0.0462207    0.0249128    …   0.0465646   0.0  0.0  0.0  0.0
 -0.0614145  -0.00094823  -0.0111628       -0.0786156   0.0  0.0  0.0  0.0
 -0.11621    -0.0522188   -0.0705224       -0.0516041   0.0  0.0  0.0  0.0
  0.0156294   0.0465735    0.092369         0.00902063  0.0  0.0  0.0  0.0
  0.0992929   0.0624509    0.0927522        0.0364857   0.0  0.0  0.0  0.0
 -0.0856861  -0.122785    -0.0563412    …   0.0395094   0.0  0.0  0.0  0.0
 -0.028133   -0.0287556   -0.0605572       -0.0588609   0.0  0.0  0.0  0.0
  0.0522207   0.0515131   -0.0540416       -0.0903071   0.0  0.0  0.0  0.

In [10]:
@assert(!(START_MARKER in indexed_words))
push!(indexed_words, START_MARKER)
word_indexes[START_MARKER] = length(indexed_words)

@assert(!(END_MARKER in indexed_words))
push!(indexed_words, END_MARKER)
word_indexes[END_MARKER] = length(indexed_words)
LL = [LL zeros(size(LL,1),2)]

300x1121 Array{Float64,2}:
  0.0529562  -0.00851202  -0.0123606    …  0.0  0.0  0.0  0.0  0.0  0.0
  0.0654598  -0.0342245   -0.0222299       0.0  0.0  0.0  0.0  0.0  0.0
  0.0661953   0.0322839    0.0655398       0.0  0.0  0.0  0.0  0.0  0.0
  0.0470722   0.0458679    0.0394772       0.0  0.0  0.0  0.0  0.0  0.0
  0.0522207  -0.0131429   -0.0866199       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0820086  -0.0462207    0.0249128    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0614145  -0.00094823  -0.0111628       0.0  0.0  0.0  0.0  0.0  0.0
 -0.11621    -0.0522188   -0.0705224       0.0  0.0  0.0  0.0  0.0  0.0
  0.0156294   0.0465735    0.092369        0.0  0.0  0.0  0.0  0.0  0.0
  0.0992929   0.0624509    0.0927522       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0856861  -0.122785    -0.0563412    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.028133   -0.0287556   -0.0605572       0.0  0.0  0.0  0.0  0.0  0.0
  0.0522207   0.0515131   -0.0540416       0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                                  

In [11]:
known_vocab = Set(indexed_words)
known_corpus = filter(corpus) do sent
    for word in sent
        if !( word in known_vocab)
            return false
        end
    end
    true
end;

In [ ]:
################LOADING Done, Now processing

In [ ]:


function collect_grams_stats(sentences)
    unigrams = counter(AbstractString)
    bigrams = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        push!(bigrams[START_MARKER], sent[1])
        for ii in 1:length(sent)-1
            push!(unigrams,sent[ii])
            push!(bigrams[sent[ii]], sent[ii+1])    
        end
        push!(unigrams,sent[end])
        push!(bigrams[sent[end]], END_MARKER)
    end
    
    
    [k=>v.map for (k,v) in bigrams], unigrams.map
end




In [ ]:
using StatsBase
# modified from https://github.com/JoFrhwld/GoodTuring.jl/blob/master/GoodTuring.jl
function simpleGoodTuring(speciesCountDict::Dict)
    speciesCountVec = collect(values(speciesCountDict))
        
    totalCounts = sum(speciesCountVec)
    cofcDict = countmap(speciesCountVec)
    r = sort(collect(keys(cofcDict)))

    N = size(r,1)
    Nr = [cofcDict[r[i]] for i in 1:N]

    p0 = haskey(cofcDict, 1.0) ? cofcDict[1.0] / totalCounts : 0.0
    
    Z = sgtZ(r,Nr)
    logr = map(log,r)
    logZ = map(log,Z)

    X = hcat(ones(N), logr)
    Y = copy(logZ )
    coefs = X\Y
    intercept = coefs[1]
    slope = coefs[2]

    useY = false
    rSmooth = Array{Float64}(N)
    for i in 1:N
        @inbounds thisr = r[i]
        
        #y = ((thisr+1.0)^(slope+1.0))/(thisr^slope)
        #The above is the much simplified form of the below (Performance identical output differs by 10^-16)
        y = (thisr+1.0) * exp(slope * log(thisr+1.0) + intercept) / exp(slope * log(thisr) + intercept)

        if !in(thisr+1, r)
            useY = true
        end

        if useY
            rSmooth[i] = y
        else
            x = (thisr+1) * cofcDict[thisr + 1]/cofcDict[thisr]
            thisNr = cofcDict[thisr]
            thisNr1 = cofcDict[thisr+1]

            t = 1.96 * ((thisr+1)^2) * (thisNr1 / thisNr^2) * (1 + (thisNr1 / thisNr))

            if abs(x-y) > t
                @inbounds rSmooth[i] = x
            else
                useY = true
                @inbounds rSmooth[i] = y
            end
        end
    end

    smoothTot = sum(Nr.*rSmooth)
    sgtProb  = (1.0 - p0) .* (rSmooth/smoothTot)
    sgtProbDict = Dict([r[i] => sgtProb[i] for i in 1:N])
    sgtDict = Dict([sp=>sgtProbDict[speciesCountDict[sp]] for sp in keys(speciesCountDict)])

    sgtDict, sgtProbDict, p0
end


function sgtZ(r::Array, Nr::Array)
    j = r
    i = [0; j[1:end-1]]
    lastK = 2*j[end] - i[end]
    k = [j[2:end]; lastK]
    Float64[(2*Nr[iter])/(k[iter]-i[iter]) for iter = 1:length(j)]
end

function simpleGoodTuring(speciesCountVec::Vector)
    sgtD = simpleGoodTuring(Dict([ii=>v for (ii,vv) in enumerate(speciesCountVec)]))
    [sgtD[ii] for ii in 1:length(speciesCountVec)]
end

In [ ]:
function katz_bigrams(bigram_freq::Dict, unigrams_freq::Dict)
    k_bigrams = Dict()
    
    for first in keys(bigram_freq)
        smoothed,_,p0 = simpleGoodTuring(bigram_freq[first])
        k_bigrams[first] = smoothed
        
        backoff_keys = setdiff(keys(unigram_freq),keys(smoothed))
        #share the p0 proability mass between them
        total = sum([unigrams_freq[key] for key in backoff_keys])
        for second in backoff_keys
            k_bigrams[first][second]=p0.*unigrams_freq[second]./total
        end
    end
    k_bigrams
end

In [ ]:
Vector

In [ ]:
function dict2mat(bigrams::Dict, word_indexes::Dict{AbstractString,Int64}, dense=False)
    mat  = (dense ? zeros: spzeros)(length(word_indexes),length(word_indexes))
    for first in keys(bigrams)
        for second in keys(bigrams[first])
            mat[word_indexes[second], word_indexes[first]] = bigrams[first][second]
        end
    end
    mat
end

In [ ]:
bigram_freq, unigram_freq = collect_grams_stats(known_corpus);
kbigrams=katz_bigrams(bigram_freq, unigram_freq)
kbigrams_mat = dict2mat(kbigrams,word_indexes,true)


In [ ]:
using Gadfly
using Distributions

In [ ]:
sent_lengths = map(length, known_corpus)
plot(x=sent_lengths, Geom.histogram)

In [ ]:
sent_length_dist = fit_mle(Gamma, sent_lengths)
plot(x=[round(rand(sent_length_dist)) for _ in 1:length(sent_lengths)], Geom.histogram)

In [ ]:
length_prob=cdf(sent_length_dist,[1.5:1.0:50.5])-cdf(sent_length_dist,[0.5:1.0:49.5])


In [ ]:
function collect_cooccur_stats(sentences)
    unioccur = counter(AbstractString)
    bioccur = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        for ii in 1:length(sent)
            push!(unioccur, sent[ii])
            for jj in 1:length(sent)
                if ii==jj
                    continue
                end
                push!(bioccur[sent[ii]], sent[jj])    
            end
        end       
    end
    
    [k=>v.map for (k,v) in bioccur], unioccur.map
    
end

In [ ]:
bioccur_freq, unioccur_freq = collect_cooccur_stats(known_corpus)

bioccur_mat = dict2mat(bioccur_freq,word_indexes,true)
bioccur_mat.+=1.0 # Add one smoothing
bioccur_mat./=sum(bioccur_mat)

unioccur = freq2prob(unioccur_freq)
unioccur_vec = Float64[word in keys(unioccurs) ? unioccurs[word] : 0.0 for word in indexed_words]

In [ ]:
unioccur_vec_smoothed = Float64[word in keys(unioccur_freq) ? unioccur_freq[word] : 0.0 for word in indexed_words]
unioccur_vec_smoothed.+=1.0 # Add one smoothing
unioccur_vec_smoothed./=sum(unioccur_vec_smoothed)

In [ ]:
sum(bioccur_mat,2)

In [ ]:
open("atis_data.jsz","w") do fh
    data = Dict([
        ("bigrams", kbigrams_mat),
        ("bioccur", bioccur_mat),
        ("unioccur", unioccur_vec),
        
        ("length_prob", length_prob),
        ("LL",LL),
        ("word_indexes", word_indexes),
        ("indexed_words", indexed_words),
        ])
    serialize(fh, data)    
end

In [ ]:
function likelyhood(sent, bigrams)
    words = split(sent)
    words = [START_MARKER; words; END_MARKER]
    
    p=1.0
    for ii in 1:length(words)-1
        p*=bigrams[words[ii]][words[ii+1]]
    end
    p
end


function select_word{S<:AbstractString,V}(unigrams::Dict{S,V})
    cutoff = rand()
    total = 0.0
    for next_word in keys(unigrams)
        total+=unigrams[next_word]
        if total>=cutoff
            return next_word
        end
    end
    assert(False, "Should never reach here") 
end

function random_walk(bigrams)
    words=[]
    cur = START_MARKER
    while(cur!=END_MARKER)
        cur = select_word(bigrams[cur])
        push!(words,cur)
    end
    words = words[1:end-1]
    join(words, " ")
end

walk =random_walk(kbigrams) 
print(walk*"\t")
print(likelyhood(walk,kbigrams))

In [ ]:
LL

Frame's Magic Mass-Sharing Co-occurance PMF, Inspired by Bengio 2003
---

In [12]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [34]:
function prepare_cases(sentence)
    Task() do
        sentence_iis = Int[word_indexes[word] for word in sentence]
        for n_givens in 1:length(sentence_iis)
            given_prob = 1.0/(n_givens)
            given_keep = rand(length(sentence_iis)).<given_prob
            given_iis::Vector{Int} =  sentence_iis[given_keep]
            cooccur_iis::Vector{Int} =  sentence_iis[~given_keep]
            produce(given_iis, cooccur_iis)
        end   
    end
end

prepare_cases (generic function with 2 methods)

In [32]:
prepare_cases

LoadError: LoadError: MethodError: `code_warntype` has no method matching code_warntype(::Symbol)
Closest candidates are:
  code_warntype(::Any, !Matched::ANY)
while loading In[32], in expression starting on line 1

In [14]:
@everywhere function loss(actual, expected)
    sum(0.5*(expected-actual).^2)
end

@everywhere function forward(x,W,b)
    tanh(W*x+b)
end


@everywhere function δ_tanh(δ_above, W, a)
    #for a is the ouput of this layer: a=tanh(z) where z is the input from layer below
    #W is matrix to move to above layer, from this one
    const da_dz = 1-a.^2 #Derivitive of a=tanh(z)
    δ(δ_above, W, da_dz)
end

@everywhere function δ_lin(δ_above, W)
    #W is matrix to move to above layer, from this one
    δ(δ_above, W, 1.0)
end
δ_lin(δ_above, W, ::Any) = δ_lin(δ_above, W)


@everywhere function δ(δ_above, W, da_dz)
    #for a=tanh(z) where z is the input from layer below, da_dz is da/dz
    #W is matrix to move to above layer, from this one
    (W'*δ_above).*da_dz
end

@everywhere function δ_output(actual, expected) 
    #Output Layer
    const M = length(actual)# ==length(expected)
    const dz = 1-actual.^2
    const δ_above = -(expected-actual)
    δ_above.*dz
end


@everywhere function feedforward_backprop(x,W,b, expected_output)
    actual_output = forward(x,W,b)
    err = loss(actual_output, expected_output)
    
    δ_top = δ_output(actual_output, expected_output)
    ΔW  = δ_top*x'
    Δb  = δ_top
    δ_bottom = δ_tanh(δ_top, W, x)
    Δx  = δ_bottom
    Δx,ΔW,Δb,err
end

@everywhere function train_one(given_iis, target_iis, C, W, b)
    given_sowe = length(given_iis)>0 ? sum([C[:,g_ii] for g_ii in given_iis]) : zeros(C[:,1])
    target = zeros(b) #just while we are testing use a one hot set rep
    for t_ii in target_iis
        target+=1.0/length(target_iis)
    end
        
    Δx,ΔW,Δb, err= feedforward_backprop(given_sowe,W,b, target)
    
    ΔC = zeros(C)
    for g_ii in given_iis
        ΔC[:, g_ii]+=Δx/length(given_iis)
    end
    ΔC,ΔW,Δb, err
end
    

In [42]:
fetch(@spawnat( 12, nworkers()))

10

In [45]:
aa=[1:20;]
aa[1:20]|> typeof

Array{Int64,1}

In [74]:
workers()

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  9
 10
 11
 12

In [90]:
mysubarray([1:100],2,10)

1:

9-element SubArray{Int64,1,Array{Int64,1},Tuple{UnitRange{Int64}},1}:
 1
 2
 3
 4
 5
 6
 7
 8
 9

9

In [91]:
@everywhere function mysubarray(xs, id=myid(), nchunks=nworkers())
    len = length(xs)
    chunk_size = div(len, nchunks+1)
    start_index = (id-2)*chunk_size + 1
    end_index = start_index+chunk_size-1
    print(start_index : end_index)
    sub(xs, start_index : end_index)
end

function train_all(training_cases,C, W, b)
    
    function train_remote()
        total_ΔC=zeros(C)
        total_ΔW=zeros(W)
        total_Δb=zeros(b)
        total_err = 0.0
        for (g_iis, t_iis) in mysubarray(training_cases)
            ΔC, ΔW, Δb,err = train_one(g_iis, t_iis, C, W, b)
            @inbounds total_ΔC+=ΔC
            @inbounds total_ΔW+=ΔW
            @inbounds total_Δb+=Δb
            total_err+=err
        end

        total_ΔC, total_ΔW, total_Δb, total_err
    end
    
    r_updates = [@spawnat(id, train_remote())  for id in workers()]
        
    
    total_ΔC=zeros(C)
    total_ΔW=zeros(W)
    total_Δb=zeros(b)
    total_err = 0.0
    for r_update in r_updates
        ΔC, ΔW, Δb, err = fetch(r_update)
        @inbounds total_ΔC+=ΔC
        @inbounds total_ΔW+=ΔW
        @inbounds total_Δb+=Δb
        total_err+=err
    end
    total_ΔC./length(training_cases),
    total_ΔW./length(training_cases),
    total_Δb./length(training_cases),
    total_err./length(training_cases)
        
end
    

train_all (generic function with 1 method)

In [95]:
function loss_and_loss_grad!(θ::Vector, grad::Vector)    
    C, W, b = unpack!(θ, CC,WW,bb)
    ΔC, ΔW, Δb, err = train_all(training_cases, C, W, b )
    pack!(grad, ΔC, ΔW, Δb)
    err/=length(training_cases)
    err
end

function loss!(θ::Vector)  
    dummy_grad = similar(θ) 
    loss_and_loss_grad!(θ, dummy_grad)
end

function loss_grad!(θ::Vector, storage::Vector) 
    #warn("loss_grad not defined")
    loss_and_loss_grad!(θ, grad)
end


loss_grad! (generic function with 1 method)

In [96]:
training_case_type = Tuple{Vector{Int64},Vector{Int64}}
training_cases = @pipe chain(map(prepare_cases, known_corpus)...)|> collect(training_case_type,_)

dEmb = 60
CC = 0.01*randn((dEmb, length(indexed_words)))
WW = 0.01*randn((length(indexed_words),dEmb))
bb = 0.001*randn((length(indexed_words)));
θ = pack(CC,WW,bb)

135641-element Array{Float64,1}:
  0.00522771 
  0.000307005
  0.0067046  
 -0.000675965
 -0.00180567 
 -0.00442763 
  0.0121721  
 -0.0119582  
  0.00766121 
  0.0129533  
  0.00404042 
 -0.00402683 
  0.0148179  
  ⋮          
  0.000327441
  0.000438831
  0.000260062
 -0.000987703
 -0.0014636  
  0.000391532
  0.000176015
 -1.44879e-5 
 -0.00016893 
 -9.09093e-5 
 -0.000514847
  0.00213931 

In [97]:
loss_and_loss_grad!(θ, similar(θ))

0.0035422304366317893

In [23]:
using Optim

opt_func = DifferentiableFunction(loss!,loss_grad!,loss_and_loss_grad!)


@time res = optimize(opt_func, θ, method=:l_bfgs, show_trace = true, store_trace = true, iterations = 2);
@printval res.f_calls 
@printval res.g_calls 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.x_converged                       
@printval res.f_converged 


Iter     Function value   Gradient norm 


LoadError: LoadError: InterruptException:
while loading In[23], in expression starting on line 155

In [ ]:
using ForwardDiff

function f(θ)
    actual = forward(unpack(θ,size(x),size(WW),size(bb))...)
    loss(actual, target)
end

# Using forwarddiff_jacobian
#g = forwarddiff_gradient(f, Float64, fadtype=:dual, n=length(pack(x,WW,bb)))
g = ForwardDiff.gradient(f)

In [ ]:
t=pack(x,WW,bb)
dg = g(t)
dxg,dWg,dbg = unpack(dg,size(x),size(WW),size(bb));

In [ ]:
axg,aWg,abg = train(x,WW,bb, target);

In [ ]:
findmax(abs(dxg.-axg))

In [ ]:
findmax(abs(dWg.-aWg))

In [ ]:
findmax(abs(dbg.-abg))

In [ ]:
[dxg axg]

In [ ]:
median(abs(dxg.-axg))